In [ ]:
from IPython.display import clear_output
# !pip install polars-permute[polars]
!pip install --pre astroquery
# !conda install -c conda-forge astroquery
# If you do not:
# !git clone https://github.com/astropy/astroquery.git
# !cd astroquery
# !python setup.py install
clear_output()

In [ ]:
from IPython.display import clear_output
# Python standard library
import time
import warnings
import os
import shutil

# Third-party software
import numpy as np
import pandas as pd
import polars as pl
#import polars_permute
from scipy.spatial import KDTree
from PIL import Image
import copy

# Astropy
from astropy import coordinates as coord
from astropy import units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.wcs import FITSFixedWarning
from astropy.stats import sigma_clipped_stats

# Photutils
from photutils.detection import IRAFStarFinder
from photutils.aperture import CircularAperture

# Astroquery. This tutorial requires 0.3.5 or greater.
import astroquery
from astroquery.skyview import SkyView
from astroquery.simbad import Simbad
Simbad.add_votable_fields('flux(V)')
from astroquery.vizier import Vizier

# Set up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Joblib for parallel processing
import joblib
from joblib import Parallel, delayed

warnings.filterwarnings('ignore')
clear_output()

In [ ]:
# Remove uncorrect rows

def remove_uncorrect_on_image(table, bound):
    #print("Before: " + str(len(table)))
    odd_lines = []
    for row in table:
        num = copy.deepcopy(row.index)
        if float(row['X'])<=0. or float(row['X'])>=bound or float(row['Y'])<=0. or float(row['Y'])>=bound:
            odd_lines.append(num)
            continue
        if float(row['Gmag'])==0. or np.isnan(float(row['Gmag'])):
            #if float(row['Bmag'])==0. or np.isnan(float(row['Bmag'])):
            odd_lines.append(num)
            continue
    # if float(row['ePos'])>32:
    #     odd_lines.append(num)
    #     continue
    
    table.remove_rows(odd_lines)
    #print("After: " + str(len(table)))
    

In [ ]:
# Convert to PIX

def separ_a(x):
	return x[0:9]

def separ_b(x):
	return x[9:19]

def add_zeros(x):
    a=x
    if len(a)<19:
        for i in range(19-len(a)):
            a = '0' + a
    return a

def w2p(image, table_stars):
    head = WCS(header=image.header)
    new_table = table_stars.copy()
    new_table.add_column(1.0, name='RA')
    new_table.add_column(1.0, name='DE')
    k=0
    for star in table_stars:
        if ~np.isnan(float(star['pmRA'])) and ~np.isnan(float(star['pmDE'])):
              pmRA = (star['pmRA'])
              pmDE = (star['pmDE'])
              world_x = (star['RA_ICRS']-pmRA*6.42/1000./3600.)
              world_y = (star['DE_ICRS']-pmDE*6.42/1000./3600.)
        else:
              world_x = (star['RA_ICRS'])
              world_y = (star['DE_ICRS'])
        x, y = head.all_world2pix(world_x, world_y, 0, tolerance = 1.0e-8, maxiter=40)
        new_table['RA_ICRS'][k] = round(float(x), 8)
        new_table['DE_ICRS'][k] = round(float(y), 8)
        new_table['RA'][k] = round(float(world_x), 8)
        new_table['DE'][k] = round(float(world_y), 8)
        new_table['Gmag'][k] = round(star['Gmag'], 8)
        k=k+1
    
    new_table.rename_column('RA_ICRS', 'X')
    new_table.rename_column('DE_ICRS', 'Y')
    
    bound = head.wcs.crpix[0]*2-1.5
    remove_uncorrect_on_image(new_table, bound)
    #new_table.remove_column('Gmag')
    
    new_table.remove_column('pmRA')
    new_table.remove_column('pmDE')
    new_table = new_table.to_pandas()
    # new data frame with split value columns
    
    new_table["Source"] = new_table["Source"].astype('string')
    new_table["Source"] = new_table["Source"].apply(add_zeros)
    new_table["Name_9"] = (new_table["Source"].apply(separ_a)).astype('int64')
    new_table["Name_10"] = (new_table["Source"].apply(separ_b)).astype('int64')
    
    new_table.pop('Source')
    new_table = pl.from_pandas(new_table)
    new_table = new_table.with_row_count(name='Row')
    
    return new_table


In [ ]:
# Find image

def get_image(ra, dec, rad, pix, cat):
    fites = SkyView.get_images(position=f"{ra}, {dec}", survey=[cat], pixels=pix, projection='Tan', radius=rad * u.deg)[0][0]
    return fites


In [ ]:
#I/197A/tic
#I/239/hip_main

def get_table(x, y, rad, m):
    v = Vizier(columns=['RA_ICRS', 'DE_ICRS', 'Gmag', 'pmRA', 'pmDE', 'Source'],#, 'ePos'], 
               column_filters={"Gmag":"<" + str(m)})
    v.ROW_LIMIT = 5000000
    result = v.query_region(coord.SkyCoord(ra=x, dec=y, 
                                           unit=(u.deg, u.deg), 
                                           frame='icrs'), 
                            radius=rad*2.*u.deg,
                            catalog=["I/345/gaia2"])
    #print(result[0])
    
    return result[0]


In [ ]:

def find_stars_on_image(image, fwhm=10.0, threshold_factor=3.0,  some_flux_threshold=50, plot_results=False):
    data = image.data
    mean, median, std = sigma_clipped_stats(data, sigma=2.0)
    finder = IRAFStarFinder(fwhm=fwhm, threshold=threshold_factor * std,
                            sharplo=0.2, sharphi=1.5,
                            roundlo=-1.0, roundhi=1.0,
                            peakmax=50000,
                            exclude_border=True, sigma_radius=2.0)
    sources = finder(data - median)

    if sources is not None:
        sources.sort('flux', reverse=True)
        bright_sources = sources[: some_flux_threshold]  # Топ-50 ярких

        if plot_results:
            positions = np.transpose((bright_sources['xcentroid'], bright_sources['ycentroid']))
            plt.imshow(data, cmap='gray', origin='lower')
            apertures = CircularAperture(positions, r=8.)
            apertures.plot(color='red', lw=1.5, alpha=0.7)
            plt.show()

        return bright_sources
    else:
        #print("No stars found!")
        return None
        

In [ ]:
# Конвертация Astropy Table в Polars DF

def qtable_to_polars(qtable):
    data_dict = {}
    for col_name in qtable.colnames:
        column = qtable[col_name]
        data_dict[col_name] = column.value if hasattr(column, 'value') else column.data
    df = pl.DataFrame(data_dict)
    if hasattr(qtable, 'meta') and qtable.meta:
        pass
        
    return df
    

In [ ]:

def create_custom_wcs(ra, dec, x, y, scale, projection='TAN'):
    wcs = WCS(naxis=2)
    wcs.wcs.crpix = [x, y]
    wcs.wcs.crval = [ra, dec]
    wcs.wcs.cdelt = [-scale, scale]
    wcs.wcs.ctype = [f"RA---{projection}", f"DEC--{projection}"]
    return wcs

def p2w(df, image, cent_ra, cent_dec, image_size, scale, shift=True):
    wcs = create_custom_wcs(cent_ra, cent_dec, image_size / 2.0, image_size / 2.0, scale)
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', FITSFixedWarning)
        ra_world, dec_world = wcs.all_pix2world(df['xcentroid'].to_numpy(), df['ycentroid'].to_numpy(), 0)

    if shift:
        head = WCS(header=image.header)
        ra_image, dec_image = head.all_pix2world(image_size / 2.0, image_size / 2.0, 0)
        delta_ra = cent_ra - ra_image
        delta_dec = cent_dec - dec_image
        ra_world = ra_world - delta_ra
        dec_world = dec_world - delta_dec
    else:
        ra_world = ra_world
        dec_world = dec_world

    ra_world = ra_world % 360
    result_df = df.with_columns([
        pl.Series('ra', ra_world),
        pl.Series('de', dec_world)
    ])

    return result_df


In [ ]:

def kdtree_match(stars, sources, tolerance=5):
    coords1 = stars.select(['RA', 'DE']).to_numpy()
    coords2 = sources.select(['ra', 'de']).to_numpy()
    tree = KDTree(coords2)
    distances, indices = tree.query(coords1, distance_upper_bound=tolerance)

    matches = []
    for i, (dist, idx) in enumerate(zip(distances, indices)):
        if idx < len(coords2) and dist <= tolerance:
            matches.append({
                'Row_star': i,
                'RA': coords1[i, 0],
                'DE': coords1[i, 1],
                'ra': coords2[idx, 0],
                'de': coords2[idx, 1],
            })
            
    return pl.DataFrame(matches)


In [ ]:
# Добавление координат IRAF, если нужно

def add_iraf_coords(table_stars, table_iraf, 
                              stars_key_col='Row', iraf_key_col='Row_star', 
                              iraf_ra_col='ra', iraf_dec_col='de'):
    
    result = table_stars.join(
        table_iraf.select([iraf_key_col, iraf_ra_col, iraf_dec_col]),
        left_on=stars_key_col,
        right_on=iraf_key_col,
        how='left'
    ).rename({iraf_ra_col: 'RA_IRAF', iraf_dec_col: 'DE_IRAF'})

    result = result.drop('Row')
    result = result.select(['X', 'Y', 'Gmag', 'RA', 'DE', 'RA_IRAF', 'DE_IRAF', 'Name_9', 'Name_10'])

    return result


In [ ]:
# Main

def gen_data(x, y, rad, pix, mag, catalog_name, scale):
    table = get_table(x, y, rad, mag)
    image = get_image(x, y, rad, pix, catalog_name)

    # Нужно для IRAF
    # sources = find_stars_on_image(image, plot_results=False)
    # sources_df = qtable_to_polars(sources)
    # sources_world = p2w(sources_df, image, x, y, pix, scale)
    
    pix_table = w2p(image, table)
    # matches = kdtree_match(pix_table, sources_world, tolerance=scale*5.) # нужно для IRAF
    # result_table = add_iraf_coords(pix_table, matches) # нужно для IRAF
    result_table = np.array(pix_table, dtype = np.float64)
    
    return image, result_table


In [ ]:

def dump(folder_name, name, data):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    if isinstance(data[0], fits.PrimaryHDU):
        image = data[0].data
    else:
        image = data[0]

    coords = data[1]

    if image.dtype != 'uint8':
        if image.max() > 255 or image.min() < 0:
            image = (image - image.min()) / (image.max() - image.min()) * 255
        image = image.astype('uint8')

    pil = Image.fromarray(image, 'RGB' if len(image.shape) == 3 and image.shape[2] == 3 else 'L')
    pil.save(os.path.join(folder_name, f"{name}.png"))
    np.savetxt(os.path.join(folder_name, f"{name}.txt"), coords)


In [ ]:

def func3(x, y, Rad, Pix, Mag, Catalog_name, Scale):
    if True:#not os.path.exists('dataset'+os.sep+name+".png"):
        #clear_output()
    
        try:
            data = gen_data(x, y, Rad, Pix, Mag, Catalog_name, Scale)
        except Exception:
            print("Image", x, y, "not found", flush=True)
            return
        dump('dataset', f'{x}_{y}', data)
        #dataset.append(data)
        print("Image:", x, y, flush=True)
        print(f"Stars on image: {len(data[1])}\n", flush=True)
      

In [ ]:
# Чистка пути

try:
    if os.path.exists('dataset') and os.path.isdir('dataset'):
        shutil.rmtree('dataset')
        print("Папка и всё её содержимое успешно удалены")
    else:
        print("Указанной папки не существует")
except PermissionError as e:
    print(f"Ошибка доступа: {e}")
except Exception as e:
    print(f"Ошибка: {e}")

In [ ]:
%%time

# Select star region
# If you have "Error HTTP TimeOut", then shorten the interval

def safe_func3(x_val, y, Rad, Pix, Mag, Catalog_name, Scale):
    try:
        return func3(x_val, y, Rad, Pix, Mag, Catalog_name, Scale)
    except Exception as e:
        print(f"Ошибка при x={x_val}, y={y}: {e}")
        return None

step = 2
n = 10
m = 10

Catalog_name = "DSS"
Rad = 1
Pix = 1024
Scale = 0.001953125 # размеря пикселя в градусах
Mag = 12

N_CORES = joblib.cpu_count()
cores_used = N_CORES


y = -step
print('\033[1m' + f"Part of positive Dec\n" + '\033[0m')
for i in range(n):
    print(f"Positive Dec iteration {i+1}/{n}")
    start_time = time.time()
    y += step
    
    results = Parallel(n_jobs=cores_used, verbose=10)(
        delayed(safe_func3)(x_val, y, Rad, Pix, Mag, Catalog_name, Scale)
        for x_val in range(0, m*step, step)
    )
    
    elapsed = time.time() - start_time
    print(f"Time expired per iteration {elapsed:.2f}s.\n")
    

y = 0
print('\033[1m' + f"Part of negative Dec\n" + '\033[0m')
for i in range(n):
    print(f"Negative Dec iteration {i+1}/{n}")
    start_time = time.time()
    y -= step
    
    results = Parallel(n_jobs=cores_used, verbose=10)(
        delayed(safe_func3)(x_val, y, Rad, Pix, Mag, Catalog_name, Scale)
        for x_val in range(0, m*step, step)
    )
    
    elapsed = time.time() - start_time
    print(f"Time expired per iteration {elapsed:.2f}s.\n")


In [ ]:
# Metadata

info_text = f"Info about catalog files:\n\nCatalog: {Catalog_name}\nField of view (rad): {Rad} sq. deg\nImage size: {Pix}×{Pix} pix\nScale: {Scale} deg in pix\nLimit mag: {Mag}\n\nColumns in text files:\n| X(pix) | Y(pix) | Gmag | RA(deg) | DE(deg) | RA_IRAF(deg) | DE_IRAF(deg) | Name_a | Name_b |"
info_filename = 'info.txt'
if not os.path.exists('info_dataset'):
    os.makedirs('info_dataset')
if info_text is not None:
    with open(os.path.join('info_dataset', info_filename), 'w', encoding="utf-8") as f:
        f.write(info_text)
            

In [ ]:
# Обработка полного каталога координат

raw_full_coords = pl.read_csv(
    "raw_coords.txt", has_header=False, separator=" ",
    new_columns=["X", "Y", "Gmag", "RA", "Dec", "RA_IRAF", "Dec_IRAF", "Name_a", "Name_b"],
    dtypes={"RA_IRAF": float, "Dec_IRAF": float, "Name_a": str, "Name_b": str}, null_values="N/A"
)

unique_full_coords = raw_full_coords.unique(subset=["Name_a", "Name_b"], keep="first")
unique_full_coords.write_parquet("full_coords.parquet")

# Если есть координаты IRAF
# iraf_coords = unique_full_coords.drop_nans()
# iraf_coords.write_parquet("coords_iraf.parquet")
